In [26]:
import torch
from torch import nn

from tokenizers import Tokenizer

import info
from data import CustomDataset, collate_fn
from layers import EmbeddingWithPosition, EncoderLayer, DecoderLayer
from sublayers import MultiHeadAttention, LayerLorm, PositionWiseFeedForward
from torch.utils.data import DataLoader
from model import Transformer

In [27]:
hyper_params = getattr(info, "base_hyper_params")
tokenizer_path = "../data/ende_WMT14_Tokenizer.json"
tokenizer = Tokenizer.from_file(tokenizer_path)
vocab_size = tokenizer.get_vocab_size()
src_train_data_path = "../data/test/test_en.txt"
tgt_train_data_path = "../data/test/test_de.txt"
training_dataset = CustomDataset(tokenizer=tokenizer, src_path=src_train_data_path, tgt_path=tgt_train_data_path)
model = Transformer(N=hyper_params["N"], vocab_size=vocab_size, pos_max_len=info.max_len,
                    d_model=hyper_params["d_model"], head=hyper_params["head"], d_k=hyper_params["d_k"],
                    d_v=hyper_params["d_v"], d_ff=hyper_params["d_ff"], drop_rate=hyper_params["dropout_rate"]).to(info.device)
with torch.no_grad():
    model.share_embedding.weight[0].fill_(0)

data tokenizing & loading: 2737it [00:00, 4361.60it/s]


In [31]:
dl = DataLoader(training_dataset, batch_size=5, collate_fn=collate_fn)
src_input, tgt_input, _, __ = next(iter(dl))

In [32]:
predict = model.forward(src_input, tgt_input)

In [35]:
predict

tensor([[ 0.0000,  0.0784,  0.0090,  ..., -0.3922,  0.2378,  0.1353],
        [ 0.0000,  0.1149,  0.0137,  ..., -0.2962,  0.3259,  0.0511],
        [ 0.0000, -0.0029, -0.0900,  ..., -0.3578,  0.3330, -0.0965],
        ...,
        [ 0.0000,  0.0998,  0.1231,  ..., -0.3502,  0.3567, -0.0336],
        [ 0.0000,  0.0449,  0.0296,  ..., -0.2167,  0.1825, -0.0781],
        [ 0.0000,  0.1212,  0.0855,  ..., -0.3149,  0.3154, -0.1464]],
       grad_fn=<ViewBackward0>)

In [36]:
a = torch.nn.Linear(5,7)

In [37]:
a.weight.shape

torch.Size([7, 5])

In [2]:
import torch

In [8]:
a = torch.randn(5,7)
b = torch.randn(5,3)

In [9]:
a_b = torch.bmm(b.unsqueeze(2), a.unsqueeze(1))

In [10]:
a_b.shape

torch.Size([5, 3, 7])

In [11]:
(torch.triu(torch.ones_like(a_b), 1) != 0)

tensor([[[False,  True,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True]],

        [[False,  True,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True]],

        [[False,  True,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True]],

        [[False,  True,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True]],

        [[False,  True,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True]]])